In [38]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import pickle
import pandas as pd
import numpy
from Viscosity_models import *
from Constants import *
from scipy import stats

In [39]:
dados = pd.read_excel("NaAlSi3O8.xlsx")
dados = dados.sort_values('temp [K]')
x = np.array(dados['temp [K]'])
y_log = np.array(dados['visc [log10 Pa.s]'])

In [40]:
rkf = RepeatedKFold(n_splits=5, n_repeats=6, random_state=2652124)

In [41]:
modelos = pickle.load(open("resultados_ajuste", "rb"))

In [42]:
medias = {}
scores = {}

for equacao, resultado in modelos.items():
    
    scores[equacao.__name__] = []
    medias[equacao.__name__] = 0
    for i, (train_index, test_index) in enumerate(rkf.split(x)):
        
        x_treino = x[train_index]
        y_treino = y_log[train_index]
        y_teste = y_log[test_index]
        x_teste = x[test_index]

        Resultado = Model(equacao).fit(y_treino, resultado.params, T = x_treino)

        y_verdadeiro = y_log[test_index]
        y_previsao = Resultado.eval(T = x_teste)
        RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared = False)
        scores[equacao.__name__].append(RMSE)
    
    medias[equacao.__name__] = np.mean(scores[equacao.__name__])  

    pickle.dump(scores, open("RMSEs", "wb"))

In [43]:
scores

{'VFT': [0.9504696407250479,
  0.8199242193417274,
  0.5160358477791226,
  0.4504490531501914,
  0.34608206803876407,
  0.683685233794045,
  0.4543726096603407,
  0.7662502909186171,
  0.7406392434109965,
  0.5766564812055786,
  0.6513965493878837,
  0.5305755679324068,
  0.8565688721386844,
  0.6677106523476422,
  0.4752021213521738,
  0.8202497980040662,
  0.7024868326273956,
  0.44948377634830294,
  0.39841248748218444,
  0.8103746925676392,
  0.8034244364116218,
  0.5068721175094402,
  0.6837917952145895,
  0.46920148574098963,
  0.7274724898609707,
  0.6627020540420699,
  0.6016556398298444,
  0.7145779983933478,
  0.6100918864838818,
  0.6668608480500555],
 'AM': [0.954230372145936,
  0.8221060919175507,
  0.5129485439395596,
  0.45946468811422325,
  0.35694590901233036,
  0.6821729395394509,
  0.45202913943671136,
  0.7709731379119339,
  0.7489775756918378,
  0.5700539694514117,
  0.653199607158755,
  0.5248575448277296,
  0.8567810638811292,
  0.66801496315824,
  0.479832553566

In [15]:
stats.shapiro(score1), stats.shapiro(score2), stats.shapiro(score3), stats.shapiro(score4)

NameError: name 'score1' is not defined

In [ ]:
stats.levene(score1, score2, score3, score4)

In [16]:
testes = {
    'ANOVA' : stats.f_oneway(score1,score2,score3, score4),
    }

print('Significância 0.05')
for teste, estatistica in testes.items():
    pvalor = estatistica.pvalue
    print(f'Para o teste {teste}, o P-valor foi: {pvalor}')

NameError: name 'score1' is not defined

In [9]:
tukey = stats.tukey_hsd(score1,score2,score3, score4)

In [10]:
print(tukey)

Tukey's HSD Pairwise Group Comparisons (95.0% Confidence Interval)
Comparison  Statistic  p-value  Lower CI  Upper CI
 (0 - 1)      0.058     0.000     0.027     0.090
 (0 - 2)      0.043     0.003     0.011     0.074
 (0 - 3)      0.058     0.000     0.027     0.090
 (1 - 0)     -0.058     0.000    -0.090    -0.027
 (1 - 2)     -0.015     0.576    -0.047     0.016
 (1 - 3)      0.000     1.000    -0.031     0.031
 (2 - 0)     -0.043     0.003    -0.074    -0.011
 (2 - 1)      0.015     0.576    -0.016     0.047
 (2 - 3)      0.015     0.576    -0.016     0.047
 (3 - 0)     -0.058     0.000    -0.090    -0.027
 (3 - 1)     -0.000     1.000    -0.031     0.031
 (3 - 2)     -0.015     0.576    -0.047     0.016



In [11]:
resultado = tukey.pvalue

In [12]:
print(resultado)

[[1.00000000e+00 2.38282112e-05 3.01905940e-03 2.38282094e-05]
 [2.38282112e-05 1.00000000e+00 5.75793792e-01 1.00000000e+00]
 [3.01905940e-03 5.75793792e-01 1.00000000e+00 5.75793780e-01]
 [2.38282094e-05 1.00000000e+00 5.75793780e-01 1.00000000e+00]]
